# **Sesión 2:** Calibración de Cámara 📷⚙️

## 0. Preparación

In [57]:
import cv2
print("OpenCV should be 4.8.0.76 Current version:", cv2.__version__)
from typing import List
import numpy as np
import imageio.v2 as imageio
import cv2
import copy
import glob
import os

OpenCV should be 4.8.0.76 Current version: 4.8.0


## **Apartado A: Calibración de cámara** (derecha e izquierda)

En este apartado se realiza la calibración de dos cámaras de un sistema estereoscópico. Para ello se trabajará con las imágenes de las carpetas ``left`` y ``right``. En primer lugar se trabajará con la carpeta ``left``. Posteriormente, deberá repetir el proceso con las imágenes en la carpeta ``right``. Ambas carpetas contienen imágenes con las que se calibrarán las cámaras. En ellas aparece el patrón de calibración en diferentes posiciones y orientaciones. Estas imágenes serán los datos de entrada.

Los pasos que deberá seguir para calibrar una cámara son:

1. Defina y ejecute el método para cargar imágenes ``load_images()``.
2. Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.
3. Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``.
4. Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)`` que proporcione las coordenadas 3D de las esquinas del patrón. El sistema de referencia utilizado deberá estar anclado al propio patrón.
5. Utilice ``cv2.calibrateCamera`` para obtener los parámetros de calibración para la cámara izquierda.

### **Tarea A.1:** Defina y ejecute el método para cargar imágenes ``load_images()``.

In [50]:
def load_images(filenames: List[str]) -> List[np.ndarray]:
    return [imageio.imread(filename) for filename in filenames]

# TODO Build a list containing the paths of all images from the left camera
imgs_path = []

for i in range(19):
    if i < 10:
        nombre = f"left_00{i}.jpg"
    else:
        nombre = f"left_0{i}.jpg"

    path = f"..\data\left\{nombre}"
    imgs_path.append(path)

imgs = load_images(imgs_path)

### **Tarea A.2:** Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.

In [51]:
# TODO Find corners with cv2.findChessboardCorners()

# Hemos comprobado que el patrón está formado por 8 filas y 6 columnas de esquinas
size = (6, 8)

# Buscamos el patrón de calibración en cada imagen original
corners = [cv2.findChessboardCorners(img, size) for img in imgs]

corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.

# Pasamos las imágenes a escala de grises para poder usar la función cv2.cornerSubPix()
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in imgs]
corners_refined = [cv2.cornerSubPix(i, cor[1], (8, 6), (-1, -1), criteria) if cor[0] else None for i, cor in zip(imgs_gray, corners_copy)]

### **Tarea A.3:** Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``

In [58]:
# Hacemos una copia para no perder las imágenes originales con la función cv2.drawChessboardCorners()
imgs_copy = copy.deepcopy(imgs)

# TODO Use cv2.drawChessboardCorners() to draw the cornes
drawed_corners = [cv2.drawChessboardCorners(img, size, corner, True) for img, corner in zip(imgs_copy, corners_refined) if corner is not None]

# TODO Show images and save when needed
def show_image(img, img_name = "imagen"):
    cv2.imshow(img_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def write_image(img, output_folder, i):
    os.makedirs(output_folder, exist_ok=True) 
    img_name = f"{output_folder}/corners_{i}.jpg"
    cv2.imwrite(img_name, img)

output_folder = "../data/output_corners"
os.makedirs(output_folder, exist_ok=True)
for i, img in enumerate(drawed_corners):
    show_image(img)
    write_image(img, output_folder, i)

### **Tarea A.4:** Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)``

In [53]:
# TODO Design the method. It should return a np.array with np.float32 elements
def get_chessboard_points(chessboard_shape, dx, dy):
    pass #remove pass when you implement the function

In [54]:
# TODO You need the points for every image, not just one (consider a list comprehension)
chessboard_points = get_chessboard_points((8, 6), 30, 30)

### **Tarea A.5:** Utilice ``cv2.calibrateCamera()`` para obtener los parámetros de calibración para la cámara izquierda

In [55]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [56]:
# TODO
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera()

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'calibrateCamera'
> Overload resolution failed:
>  - calibrateCamera() missing required argument 'objectPoints' (pos 1)
>  - calibrateCamera() missing required argument 'objectPoints' (pos 1)


In [ ]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

### **Pregunta A.1:** Repita el proceso (carga de imágenes, detección y comprobación de esquinas, etc.) para la cámara derecha.

In [ ]:
# TODO Homework

### **Pregunta A.2:** Diferencias entre cv2.findChessboardCorners() y cv2.cornerSubPix()

In [ ]:
# TODO Homework

### **Pregunta A.3:** Número mínimo de imágenes necesarias para calibrar.

In [ ]:
# TODO Homework

## **Apartado B: Corrección de distorsión** (ojo de pez)

En este apartado se trabajará en la corrección de la distorsión debido a lentes de ojo de pez. Primero se calibrará una cámara con este tipo de lente, utilizando las imágenes de la carpeta ``fisheye``. Posteriormente se utilizarán estos parámetros de calibración para corregir la distorsión de una de las imágenes de calibración.

Los pasos que deberá seguir para calibrar una cámara con distorsión de ojo de pez son:

1. Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``.
2. Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.
3. Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero.
4. Defina los argumentos para la función de calibración.
5. Calibre con ``cv2.fisheye.calibrate()``

### **Tarea B.1:** Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``

In [ ]:
# TODO Build a list containing the paths of all images from the fisheye camera and load images
fisheye_imgs_path = []
fisheye_imgs = load_images()

### **Tarea B.2:** Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.

In [ ]:
imgs_corners = []
# Parameters for cv2.cornerSubPix()
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)

#TODO Complete the required parts of the loop
for img in fisheye_imgs:
    
    # TODO parse arguments to cv2.findChessboardCorners()
    corners = cv2.findChessboardCorners()
    
    # TODO convert image to grayscale to use cv2.cornerSubPix()
    gray_img = cv2.cvtColor()
    refined_corners = cv2.cornerSubPix(gray_img, corners, (3,3), (-1,-1), subpix_criteria)

    # TODO append only those refined_corners with proper detections
    imgs_corners.append(refined_corners)

### **Tarea B.3:** Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero

In [ ]:
# TODO Define the chessboard dimensions and the lenght of the squares (in [mm])
chessboard_dims = (None, None)
length = None
# TODO You need the points for every image, not just one (consider a list comprehension)
fisheye_chessboard_points = get_chessboard_points(chessboard_dims, length, length)

### **Tarea B.4:** Defina los argumentos para la calibración

In [ ]:
# Parameters for cv2.fisheye.calibrate()
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW
intrinsics = np.zeros((3, 3))
distortion = np.zeros((4, 1))
rotations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners]
traslations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners]


### **Tarea B.5:** Calibración

In [ ]:
rms, _, _, _, _ = \
cv2.fisheye.calibrate(fisheye_chessboard_points, imgs_corners, gray_img.shape[::-1], intrinsics, distortion, rotations, traslations, calibration_flags, subpix_criteria)

In [ ]:
# Show intrinsic matrix and distortion coefficients values
print(intrinsics)
print(distortion)

### **Pregunta B.1:** Corrija la distorsión de las 2 primeras imágenes de la carpeta ``fisheye``

In [ ]:
# TODO Search in the documentation to define 'dim'
dim = None
map1, map2 = cv2.fisheye.initUndistortRectifyMap(intrinsics, distortion, np.eye(3), intrinsics, dim, cv2.CV_16SC2)

In [ ]:
# TODO Homework: correct distortion using cv2.remap()